In [1]:
%%configure -f
{
    "driverMemory":"4G",
    "executorMemory":"20G",
    "executorCores":8,
    "numExecutors":10
}

In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
14,application_1625762687750_0063,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime, timedelta

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
base="s3://midgar-aws-workspace/prod/shinra/telescope/base/l1_banner_events_base_table/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
outpath='s3a://pai-gm-midgar-shared-ml-team/pengcheng/dampen_intent/ngram/days=14/'
outpath

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://pai-gm-midgar-shared-ml-team/pengcheng/dampen_intent/ngram/days=14/'

In [26]:
df=spark.read.parquet(base).filter(F.col('dt').between('2021-06-15', '2021-06-30')).drop('dt', 'l1_category_name').na.fill(0)\
.filter(F.col('banner_clicks')+F.col('icon_clicks')+F.col('sales')>0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df.show(2,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+-------------+-----------+-----+
|customer_id|cma_id|banner_clicks|icon_clicks|sales|
+-----------+------+-------------+-----------+-----+
|458206492  |6797  |0            |0          |2    |
|1266843017 |357   |0            |0          |1    |
+-----------+------+-------------+-----------+-----+
only showing top 2 rows

In [28]:
agg=df.groupBy('customer_id').agg(F.collect_list(F.col('cma_id')).alias('id_seq'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
agg.coalesce(200).write.mode('overwrite').parquet(outpath+"/banner_history")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
agg=spark.read.parquet(outpath+"/banner_history")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#agg.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
agg.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: string (nullable = true)
 |-- id_seq: array (nullable = true)
 |    |-- element: string (containsNull = true)

In [33]:
from pyspark.ml.feature import NGram

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
ngram = NGram(n=2, inputCol="id_seq", outputCol="ngrams")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
ngdf = ngram.transform(agg)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#ngdf.select("ngrams").show(5, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
ngdf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: string (nullable = true)
 |-- id_seq: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngrams: array (nullable = true)
 |    |-- element: string (containsNull = false)

In [38]:
def extract_ngram(seq, cma_id):
    
    ret=[]
    for x in seq:
        pair=x.split(' ')
        if pair[0]==cma_id:
            ret.append(pair)
    return ret

schema = ArrayType(StructType([
    StructField("pre", StringType(), False),
    StructField("nxt", StringType(), False)
]))

def _extract_ngram_udf(cma_id):
    return F.udf(lambda x: extract_ngram(x, cma_id), schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
check_id='6843'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
ngdf2=ngdf.withColumn('tar_ngrams', _extract_ngram_udf(check_id)("ngrams"))\
.select('customer_id', 'id_seq', F.explode(F.col('tar_ngrams')).alias('tar_ngram'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#ngdf2.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
agg_ngdf=ngdf2.groupBy('tar_ngram').agg(F.count('customer_id').alias('count'))\
.filter(F.col('tar_ngram.pre') == check_id)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
agg_ngdf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- tar_ngram: struct (nullable = true)
 |    |-- pre: string (nullable = false)
 |    |-- nxt: string (nullable = false)
 |-- count: long (nullable = false)

In [44]:
outpath+"/agg_ngdf"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://pai-gm-midgar-shared-ml-team/pengcheng/dampen_intent/ngram/days=14//agg_ngdf'

In [45]:
agg_ngdf.write.mode('overwrite').parquet(outpath+"/agg_ngdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
agg_ngdf=spark.read.parquet(outpath+"/agg_ngdf").withColumn('ratio', F.col('count')/F.sum('count').over(Window.partitionBy()))\
.sort(F.col('ratio').desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
agg_ngdf.filter(F.col('tar_ngram.nxt')==check_id).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+--------------------+
|tar_ngram   |count  |ratio               |
+------------+-------+--------------------+
|[6843, 6843]|1881150|0.059963642170874146|
+------------+-------+--------------------+

In [48]:
agg_ngdf.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------+--------------------+
|tar_ngram   |count  |ratio               |
+------------+-------+--------------------+
|[6843, 5]   |5681172|0.18109335508555374 |
|[6843, 357] |4363458|0.13908983023131496 |
|[6843, 6841]|2284477|0.07282011608621963 |
|[6843, 352] |2108864|0.0672222663174326  |
|[6843, 6843]|1881150|0.059963642170874146|
|[6843, 6842]|1709133|0.05448041869836677 |
|[6843, 6821]|1332627|0.042478892472820086|
|[6843, 6850]|1042141|0.033219344558167586|
|[6843, 6797]|1024838|0.03266779316647493 |
|[6843, 443] |915490 |0.029182210228324998|
|[6843, 435] |789253 |0.02515827258554019 |
|[6843, 11]  |772141 |0.024612809520485307|
|[6843, 6808]|656512 |0.020927013076514327|
|[6843, 358] |614513 |0.019588250613375   |
|[6843, 6851]|505156 |0.016102380790723812|
|[6843, 420] |491145 |0.01565576537437949 |
|[6843, 40]  |476578 |0.015191426871068687|
|[6843, 6809]|386272 |0.012312827785465219|
|[6843, 6852]|325862 |0.010387195260923047|
|[6843, 6810]|293184 |0.00934554